# EDA: Анализ всех данных проекта

**Проект:** Рекомендательная система для e-commerce

## Цель анализа
Провести исследовательский анализ данных (EDA) всех файлов:
- `events.csv` - события пользователей
- `item_properties_part1.csv` + `item_properties_part2.csv` - свойства товаров
- `category_tree.csv` - дерево категорий


In [1]:
# Импорты
import sys
import os
import logging
import datetime
from pathlib import Path

# Определяем корень проекта и добавляем его в PYTHONPATH
try:
    # Для .py файла используем __file__
    notebook_dir = Path(__file__).parent
except NameError:
    # Для Jupyter notebook используем текущую рабочую директорию
    notebook_dir = Path.cwd()
    # Если запущено из notebooks/, берем родителя
    if notebook_dir.name == "notebooks":
        project_root = notebook_dir.parent
    else:
        # Если запущено из корня проекта
        project_root = notebook_dir
else:
    project_root = notebook_dir.parent if notebook_dir.name == "notebooks" else notebook_dir

# Добавляем корень проекта в PYTHONPATH (не src, а родительскую директорию src!)
# Это необходимо, чтобы импорты вида "from src.recsys.data.events import load_events" работали
project_root = project_root.resolve()
if (project_root / "src").exists():
    sys.path.insert(0, str(project_root))
else:
    raise RuntimeError(f"Директория src не найдена в проекте: {project_root / 'src'}")

import polars as pl

from src.recsys.data.events import load_events
from src.recsys.data.item_properties import load_item_properties, load_category_tree
from src.recsys.data.validation import (
    validate_events,
    validate_item_properties,
    validate_category_tree,
)
from src.recsys.utils.logging import (
    get_log_file_path,
    configure_polars_logging,
    setup_notebook_logging,
)

# Настройка логирования
# Можно отключить в проде через переменную окружения EDA_LOG_LEVEL=WARNING или ERROR
log_level = os.getenv("EDA_LOG_LEVEL", "INFO").upper()

# Путь к директории логов
logs_dir = project_root / "logs"
logs_dir.mkdir(exist_ok=True)

# Создаём единый timestamp для всего запуска
run_timestamp = datetime.datetime.now()
log_file_path = get_log_file_path("eda_processing", logs_dir, run_timestamp)

# Настройка логирования через утилиту
logger = logging.getLogger(__name__)
setup_notebook_logging(
    logger=logger,
    log_file_path=log_file_path,
    log_level=log_level,
    log_to_console=True,
    log_to_file=True,
)

# Настройка отображения Polars
configure_polars_logging()

logger.info(f"Время запуска: {run_timestamp}")
logger.info(f"Лог файл: {log_file_path}")

# Путь к данным относительно местоположения скрипта
# Работает и при запуске из корня проекта, и из директории notebooks
# script_dir и project_root уже определены выше, просто используем их
data_dir = project_root / "data" / "raw"

2025-12-21 17:26:38 - INFO - Время запуска: 2025-12-21 17:26:38.316900
2025-12-21 17:26:38 - INFO - Лог файл: /home/mle-user/mle-pr-final/logs/eda_processing_2025-12-21_17-26-38.log


## 1. Загрузка всех данных


In [2]:
# Загрузка всех трёх источников данных проекта
# Это основной шаг EDA - нужно убедиться, что все данные загружаются корректно

# 1. СОБЫТИЯ ПОЛЬЗОВАТЕЛЕЙ (events.csv)
# Содержит: visitorid, itemid, event (view/addtocart/transaction), timestamp
# Это основная информация для collaborative filtering (ALS модель)
events = load_events(data_dir)
logger.info(f"Загружено сырых событий: {len(events):,}")

# Валидация и очистка событий
events, events_stats = validate_events(events, log_stats=True)
# Преобразуем timestamp обратно в Datetime для удобства анализа
events = events.with_columns(
    pl.from_epoch(pl.col("timestamp"), time_unit="ms").alias("timestamp")
)
logger.info(f"Валидированных событий: {len(events):,}")

# 2. СВОЙСТВА ТОВАРОВ (item_properties_part1.csv + item_properties_part2.csv)
item_properties = load_item_properties(data_dir)
logger.info(f"Загружено сырых свойств: {len(item_properties):,}")

# Валидация и очистка свойств товаров
item_properties, props_stats = validate_item_properties(item_properties, log_stats=True)
logger.info(f"Валидированных свойств: {len(item_properties):,}")

# 3. ДЕРЕВО КАТЕГОРИЙ (category_tree.csv)
category_tree = load_category_tree(data_dir)
logger.info(f"Загружено сырых категорий: {len(category_tree):,}")

# Валидация и очистка дерева категорий
category_tree, tree_stats = validate_category_tree(category_tree, log_stats=True)
logger.info(f"Валидированных категорий: {len(category_tree):,}")



2025-12-21 17:26:38 - INFO - Загружено сырых событий: 2,756,101
  Удалено записей с некорректными ID: 3
2025-12-21 17:26:39 - INFO - Валидированных событий: 2,755,638
2025-12-21 17:26:40 - INFO - Загружено сырых свойств: 20,275,902
  Удалено записей с некорректными ID: 45
2025-12-21 17:26:43 - INFO - Валидированных свойств: 20,275,857
2025-12-21 17:26:43 - INFO - Загружено сырых категорий: 1,669
  Удалено записей с некорректными ID: 1
2025-12-21 17:26:43 - INFO - Валидированных категорий: 1,668


## 2. Анализ событий (events.csv)

### 2.1. Общая статистика


In [3]:
logger.info("ОБЩАЯ СТАТИСТИКА ПО СОБЫТИЯМ")

# БАЗОВЫЕ МЕТРИКИ РАЗМЕРНОСТИ ДАННЫХ
# Эти числа критичны для понимания масштаба проблемы рекомендаций
n_events = len(events)                                    # Общее количество взаимодействий
n_users = events.select("user_id").n_unique()          # Уникальных пользователей
n_items = events.select("item_id").n_unique()             # Уникальных товаров

logger.info(f"Всего событий: {n_events:,}")
logger.info(f"Уникальных пользователей: {n_users:,}")
logger.info(f"Уникальных товаров: {n_items:,}")

# СРЕДНИЕ ПОКАЗАТЕЛИ АКТИВНОСТИ
logger.info(f"Среднее событий на пользователя: {n_events / n_users:.2f}")
logger.info(f"Среднее событий на товар: {n_events / n_items:.2f}")

# РАСПРЕДЕЛЕНИЕ ПО ТИПАМ СОБЫТИЙ
event_counts = events.group_by("event").agg([
    pl.len().alias("count"),
    (pl.len() / n_events * 100).alias("percentage")
]).sort("count", descending=True)

logger.info("РАСПРЕДЕЛЕНИЕ ПО ТИПАМ СОБЫТИЙ:")
logger.info(f"{event_counts}")

# ВРЕМЕННОЙ ДИАПАЗОН ДАННЫХ
# Важно для temporal split (разделение на train/test по времени)
# Позволяет оценить, какой период данных доступен для обучения
events_with_dt = events.with_columns([
    pl.col("timestamp").dt.year().alias("year"),
    pl.col("timestamp").dt.month().alias("month"),
    pl.col("timestamp").dt.day().alias("day"),
])

min_ts = events_with_dt.select(pl.col("timestamp").min()).item()
max_ts = events_with_dt.select(pl.col("timestamp").max()).item()
days_span = (max_ts - min_ts).days

logger.info("ВРЕМЕННОЙ ДИАПАЗОН:")
logger.info(f"   От: {min_ts}")
logger.info(f"   До: {max_ts}")
logger.info(f"   Период: {days_span} дней (~{days_span/30:.1f} месяцев)")

2025-12-21 17:26:43 - INFO - ОБЩАЯ СТАТИСТИКА ПО СОБЫТИЯМ
2025-12-21 17:26:43 - INFO - Всего событий: 2,755,638
2025-12-21 17:26:43 - INFO - Уникальных пользователей: 1,407,579
2025-12-21 17:26:43 - INFO - Уникальных товаров: 235,061
2025-12-21 17:26:43 - INFO - Среднее событий на пользователя: 1.96
2025-12-21 17:26:43 - INFO - Среднее событий на товар: 11.72
2025-12-21 17:26:43 - INFO - РАСПРЕДЕЛЕНИЕ ПО ТИПАМ СОБЫТИЙ:
2025-12-21 17:26:43 - INFO - shape: (3, 3)
┌─────────────┬─────────┬────────────┐
│ event       ┆ count   ┆ percentage │
│ ---         ┆ ---     ┆ ---        │
│ str         ┆ u32     ┆ f64        │
╞═════════════╪═════════╪════════════╡
│ view        ┆ 2664215 ┆ 96.682329  │
│ addtocart   ┆ 68966   ┆ 2.502724   │
│ transaction ┆ 22457   ┆ 0.814947   │
└─────────────┴─────────┴────────────┘
2025-12-21 17:26:44 - INFO - ВРЕМЕННОЙ ДИАПАЗОН:
2025-12-21 17:26:44 - INFO -    От: 2015-05-03 03:00:04.384000
2025-12-21 17:26:44 - INFO -    До: 2015-09-18 02:59:47.788000
2025-12-

### Анализ результатов

**Масштаб данных:**
- **1,407,579 пользователей** и **235,061 товаров** - большая размерность задачи
- **Среднее 1.96 событий на пользователя** - **КРИТИЧЕСКИ НИЗКАЯ активность!**
  - Это указывает на экстремальную cold-start проблему
  - У большинства пользователей очень мало взаимодействий
- **Среднее 11.72 событий на товар** - более равномерное распределение, чем по пользователям

**Распределение событий:**
- **96.68% views** - слабый сигнал интереса
- **2.50% addtocart** - средний сигнал (важно для e-commerce!)
- **0.81% transaction** - сильный сигнал (конверсия)

**Выводы для модели:**
1. **Огромный дисбаланс событий** - необходимы правильные веса:
   - `view = 1.0` (слабый сигнал)
   - `addtocart = 4.0` (средний сигнал)
   - `transaction = 8.0` (сильный сигнал)
2. **Временной диапазон:** ~4.6 месяцев данных
   - Достаточно для обучения, но короткий период
   - При `test_size=0.15`: ~3.9 месяцев train, ~0.7 месяцев test
3. **Cold-start критична** - нужен обязательный fallback механизм для новых/малоактивных пользователей



### 2.2. Разреженность данных (Sparsity Analysis)


In [4]:
# АНАЛИЗ РАЗРЕЖЕННОСТИ ДАННЫХ - КЛЮЧЕВОЙ МЕТРИКА ДЛЯ РЕКОМЕНДАТЕЛЬНЫХ СИСТЕМ
# Разреженность показывает, насколько мало взаимодействий относительно возможных
# Высокая разреженность = сложная задача для collaborative filtering

# СТАТИСТИКА АКТИВНОСТИ ПОЛЬЗОВАТЕЛЕЙ
# Показывает распределение событий по пользователям (перцентили)
user_stats = events.group_by("user_id").agg([
    pl.len().alias("n_events"),              # Количество событий пользователя
    pl.n_unique("item_id").alias("n_items"),  # Количество уникальных товаров
    pl.n_unique("event").alias("n_event_types"),  # Типы событий
]).sort("n_events", descending=False)

user_percentiles = user_stats.select([
    pl.col("n_events").quantile(0.5).alias("median"),  # Медиана
    pl.col("n_events").quantile(0.75).alias("p75"),    # 75-й перцентиль
    pl.col("n_events").quantile(0.90).alias("p90"),    # 90-й перцентиль
    pl.col("n_events").quantile(0.95).alias("p95"),    # 95-й перцентиль
    pl.col("n_events").quantile(0.99).alias("p99"),    # 99-й перцентиль
])

logger.info("РАЗРЕЖЕННОСТЬ ПО ПОЛЬЗОВАТЕЛЯМ:")
logger.info(f"   Медиана событий на пользователя: {user_percentiles.select('median').item():.1f}")
logger.info(f"   P75: {user_percentiles.select('p75').item():.1f}")
logger.info(f"   P90: {user_percentiles.select('p90').item():.1f}")
logger.info(f"   P95: {user_percentiles.select('p95').item():.1f}")
logger.info(f"   P99: {user_percentiles.select('p99').item():.1f}")

# СТАТИСТИКА ПОПУЛЯРНОСТИ ТОВАРОВ
# Показывает, как распределены события по товарам
item_stats = events.group_by("item_id").agg([
    pl.len().alias("n_events"),              # Количество событий товара
    pl.n_unique("user_id").alias("n_users"),  # Количество уникальных пользователей
]).sort("n_events", descending=False)

item_percentiles = item_stats.select([
    pl.col("n_events").quantile(0.5).alias("median"),  # Медиана
    pl.col("n_events").quantile(0.90).alias("p90"),    # 90-й перцентиль
    pl.col("n_events").quantile(0.99).alias("p99"),    # 99-й перцентиль
])

logger.info("РАЗРЕЖЕННОСТЬ ПО ТОВАРАМ:")
logger.info(f"   Медиана событий на товар: {item_percentiles.select('median').item():.1f}")
logger.info(f"   P90: {item_percentiles.select('p90').item():.1f}")
logger.info(f"   P99: {item_percentiles.select('p99').item():.1f}")

# ОБЩАЯ РАЗРЕЖЕННОСТЬ МАТРИЦЫ ВЗАИМОДЕЙСТВИЙ
# Показывает, какая доля всех возможных взаимодействий отсутствует
# Формула: sparsity = (1 - фактические / возможные) * 100%
possible_interactions = n_users * n_items  # Все возможные пары (пользователь, товар)
actual_interactions = n_events              # Фактические взаимодействия
sparsity = (1 - actual_interactions / possible_interactions) * 100

logger.info("МАТРИЦА РАЗРЕЖЕННОСТИ:")
logger.info(f"   Возможных взаимодействий: {possible_interactions:,}")
logger.info(f"   Фактических взаимодействий: {actual_interactions:,}")
logger.info(f"   Разреженность: {sparsity:.4f}%")

2025-12-21 17:26:44 - INFO - РАЗРЕЖЕННОСТЬ ПО ПОЛЬЗОВАТЕЛЯМ:
2025-12-21 17:26:44 - INFO -    Медиана событий на пользователя: 1.0
2025-12-21 17:26:44 - INFO -    P75: 2.0
2025-12-21 17:26:44 - INFO -    P90: 3.0
2025-12-21 17:26:44 - INFO -    P95: 5.0
2025-12-21 17:26:44 - INFO -    P99: 13.0
2025-12-21 17:26:44 - INFO - РАЗРЕЖЕННОСТЬ ПО ТОВАРАМ:
2025-12-21 17:26:44 - INFO -    Медиана событий на товар: 3.0
2025-12-21 17:26:44 - INFO -    P90: 25.0
2025-12-21 17:26:44 - INFO -    P99: 143.0
2025-12-21 17:26:44 - INFO - МАТРИЦА РАЗРЕЖЕННОСТИ:
2025-12-21 17:26:44 - INFO -    Возможных взаимодействий: 330,866,927,319
2025-12-21 17:26:44 - INFO -    Фактических взаимодействий: 2,755,638
2025-12-21 17:26:44 - INFO -    Разреженность: 99.9992%


### Анализ разреженности

**Активность пользователей:**
- **Медиана = 1.0** - у 50% пользователей **ТОЛЬКО 1 событие**!
- **P75 = 2.0** - у 75% пользователей максимум 2 события
- **P99 = 13.0** - только 1% пользователей имеют 13+ событий

**Вывод:** **71.15% пользователей имеют только 1 событие** → экстремальная cold-start проблема!

**Популярность товаров:**
- **Медиана = 3.0** события на товар
- **P90 = 25.0** - топ 10% товаров имеют 25+ событий
- **P99 = 143.0** - топ 1% товаров очень популярны

**Вывод:** Распределение длинный хвост - есть очень популярные и очень редкие товары.
- Popular fallback будет работать хорошо (есть топ товары)
- **Риск:** Coverage может быть низкой (много редких товаров не попадут в рекомендации)

**Общая разреженность матрицы:**
- **99.9992% разреженность** - **ЭКСТРЕМАЛЬНО высокая!**
- Из 330+ миллиардов возможных взаимодействий есть только 2.7 миллиона

**Выводы для модели:**
1. **ALS модель подходит** (работает со sparse матрицами), но необходимы:
   - Агрессивные фильтры (min_user_interactions ≥ 32)
   - Обязательный fallback для cold users
   - Возможно, комбинация с content-based подходами
2. **Обязателен popular items fallback** для пользователей с 1-2 событиями
3. Нужно следить за **Coverage** метрикой - много редких товаров могут не попасть в рекомендации



## 3. Анализ свойств товаров (item_properties)

### 3.1. Общая статистика


In [5]:
logger.info("ОБЩАЯ СТАТИСТИКА ПО СВОЙСТВАМ ТОВАРОВ")

# БАЗОВЫЕ МЕТРИКИ МЕТАДАННЫХ ТОВАРОВ
# Свойства товаров - источник для content-based рекомендаций
n_properties = len(item_properties)                                    # Всего записей свойств
n_items_with_props = item_properties.select("item_id").n_unique()      # Товаров с метаданными
n_unique_properties = item_properties.select("property").n_unique()   # Типов свойств

logger.info(f"Всего записей свойств: {n_properties:,}")
logger.info(f"Уникальных товаров с свойствами: {n_items_with_props:,}")
logger.info(f"Уникальных типов свойств: {n_unique_properties}")

# ТОП САМЫХ ЧАСТЫХ СВОЙСТВ
# Показывает, какие свойства присутствуют у большинства товаров
# Важно для feature engineering - эти свойства можно использовать в модели
top_properties = item_properties.group_by("property").agg([
    pl.len().alias("count"),                # Количество записей
    pl.n_unique("item_id").alias("n_items"), # Товаров с этим свойством
]).sort("count", descending=True).head(20)

logger.info("ТОП-20 САМЫХ ЧАСТЫХ СВОЙСТВ:")
logger.info(f"{top_properties}")

# СПЕЦИАЛЬНЫЕ СВОЙСТВА (ИЗВЕСТНАЯ СЕМАНТИКА)
# categoryid - категория товара (критично для content-based!)
# available - доступность товара (важно для рекомендаций)
special_props = item_properties.filter(
    pl.col("property").is_in(["categoryid", "available"])
)

if len(special_props) > 0:
    logger.info("СПЕЦИАЛЬНЫЕ СВОЙСТВА:")
    logger.info(f"{special_props.group_by('property').agg([pl.len().alias('count'), pl.n_unique('item_id').alias('n_items')])}")

2025-12-21 17:26:44 - INFO - ОБЩАЯ СТАТИСТИКА ПО СВОЙСТВАМ ТОВАРОВ
2025-12-21 17:26:46 - INFO - Всего записей свойств: 20,275,857
2025-12-21 17:26:46 - INFO - Уникальных товаров с свойствами: 417,052
2025-12-21 17:26:46 - INFO - Уникальных типов свойств: 1104
2025-12-21 17:26:46 - INFO - ТОП-20 САМЫХ ЧАСТЫХ СВОЙСТВ:
2025-12-21 17:26:46 - INFO - shape: (20, 3)
┌────────────┬─────────┬─────────┐
│ property   ┆ count   ┆ n_items │
│ ---        ┆ ---     ┆ ---     │
│ str        ┆ u32     ┆ u32     │
╞════════════╪═════════╪═════════╡
│ 888        ┆ 3000397 ┆ 417052  │
│ 790        ┆ 1790515 ┆ 417052  │
│ available  ┆ 1503638 ┆ 417052  │
│ categoryid ┆ 788213  ┆ 417052  │
│ 6          ┆ 631453  ┆ 409064  │
│ 283        ┆ 597418  ┆ 417052  │
│ 776        ┆ 574219  ┆ 407304  │
│ 678        ┆ 481965  ┆ 417018  │
│ 364        ┆ 476485  ┆ 417052  │
│ 202        ┆ 448937  ┆ 414216  │
│ 839        ┆ 417238  ┆ 396643  │
│ 917        ┆ 417226  ┆ 416170  │
│ 764        ┆ 417052  ┆ 417052  │
│ 112   

### Анализ свойств товаров

**Метаданные:**
- **20,275,857 записей свойств** - богатый источник метаданных
- **417,052 товаров с метаданными** - на **77% больше**, чем товаров в событиях (235,061)!
- **1,104 типов свойств** - большое разнообразие характеристик

**Вывод:** Много товаров есть в каталоге, но не было взаимодействий (long tail). Это указывает на потенциал для улучшения рекомендаций через content-based подходы.

**Топ свойства:**
- Свойства с числовыми ID (888, 790, 6, 283...) - возможно, коды брендов, цен, характеристик
- Все топ-20 свойств присутствуют у 400K+ товаров

**Специальные свойства:**
- **categoryid:** 788,213 записей, **417,052 товара** (все товары с метаданными имеют категорию!)
- **available:** 1,503,638 записей, 417,052 товара (много временных изменений доступности)

**Выводы для модели:**
1. **Категории доступны для ВСЕХ товаров с метаданными** - отлично для content-based подходов!
2. Можно использовать категории для:
   - Re-ranking рекомендаций
   - Cold-start стратегий (новые пользователи → популярные категории)
   - Content-based фильтрации
3. Свойства с ID могут быть полезны для feature engineering (требует дополнительного анализа)



### 3.2. Категории товаров (categoryid)


In [6]:
# Извлечение категорий из свойств товаров
categories_from_props = item_properties.filter(
    pl.col("property") == "categoryid"
).select([
    pl.col("item_id").alias("item_id"),
    pl.col("value").cast(pl.Int64).alias("categoryid"),
])

logger.info("СТАТИСТИКА ПО КАТЕГОРИЯМ:")
logger.info(f"   Товаров с категориями: {categories_from_props.select('item_id').n_unique():,}")
logger.info(f"   Уникальных категорий: {categories_from_props.select('categoryid').n_unique()}")

# Распределение товаров по категориям
cat_dist = categories_from_props.group_by("categoryid").agg([
    pl.len().alias("n_items"),
]).sort("n_items", descending=True)

logger.info("РАСПРЕДЕЛЕНИЕ ТОВАРОВ ПО КАТЕГОРИЯМ:")
logger.info(f"   Медиана товаров на категорию: {cat_dist.select(pl.col('n_items').quantile(0.5)).item()}")
logger.info(f"   Максимум: {cat_dist.select(pl.col('n_items').max()).item()}")
logger.info("   Топ-10 категорий по количеству товаров:")
logger.info(f"{cat_dist.head(10)}")

2025-12-21 17:26:47 - INFO - СТАТИСТИКА ПО КАТЕГОРИЯМ:
2025-12-21 17:26:47 - INFO -    Товаров с категориями: 417,052
2025-12-21 17:26:47 - INFO -    Уникальных категорий: 1242
2025-12-21 17:26:47 - INFO - РАСПРЕДЕЛЕНИЕ ТОВАРОВ ПО КАТЕГОРИЯМ:
2025-12-21 17:26:47 - INFO -    Медиана товаров на категорию: 110.0
2025-12-21 17:26:47 - INFO -    Максимум: 26890
2025-12-21 17:26:47 - INFO -    Топ-10 категорий по количеству товаров:
2025-12-21 17:26:47 - INFO - shape: (10, 2)
┌────────────┬─────────┐
│ categoryid ┆ n_items │
│ ---        ┆ ---     │
│ i64        ┆ u32     │
╞════════════╪═════════╡
│ 1147       ┆ 26890   │
│ 546        ┆ 24885   │
│ 1613       ┆ 21126   │
│ 491        ┆ 19998   │
│ 1404       ┆ 18217   │
│ 1120       ┆ 18038   │
│ 342        ┆ 17231   │
│ 1277       ┆ 14990   │
│ 1167       ┆ 13509   │
│ 282        ┆ 11964   │
└────────────┴─────────┘


### Анализ категорий товаров

**Статистика:**
- **417,052 товаров с категориями** - **100% покрытие** для товаров с метаданными!
- **1,242 уникальных категорий** - богатое разнообразие

**Распределение:**
- **Медиана: 110 товаров/категория** - средний размер категории
- **Максимум: 26,890 товаров** (категория 1147) - очень большая категория!
- Топ-10 категорий содержат от 11K до 27K товаров

**Выводы:**
1. **Неравномерное распределение** - есть мега-категории (26K+ товаров) и маленькие категории
2. **Для рекомендаций нужно учитывать популярность категории** - крупные категории более важны
3. **Популярные категории можно использовать для cold-start fallback** - рекомендуя товары из топ-10 категорий для новых пользователей

**Применение для модели:**
- Re-ranking с учётом популярности категории
- Cold-start: товары из популярных категорий для новых пользователей
- Diversity constraint: ограничивать количество товаров из одной категории



## 4. Анализ дерева категорий (category_tree)


In [7]:
# АНАЛИЗ ДЕРЕВА КАТЕГОРИЙ - ИЕРАРХИЧЕСКАЯ СТРУКТУРА
# category_tree.csv содержит иерархию: каждая категория имеет parentid (кроме корневых)
# Это позволяет понять структуру каталога и использовать для content-based подходов

n_categories = len(category_tree)
root_categories = category_tree.filter(pl.col("parent_id").is_null()).select("category_id").n_unique()

logger.info(f"Всего категорий в дереве: {n_categories}")
logger.info(f"Корневых категорий (без родителя): {root_categories}")

# ГЛУБИНА ДЕРЕВА (ПОЛНЫЙ РЕКУРСИВНЫЙ АНАЛИЗ)
# Рекурсивно вычисляем уровень каждой категории (глубину от корня)
# Важно: упрощённый анализ (только 2 уровня) неверен - нужна рекурсия!
def calculate_depth(category_tree_df):
    """Вычисляет глубину каждой категории рекурсивно.
    
    Использует мемоизацию (кеш) для оптимизации - каждая категория
    вычисляется только один раз, несмотря на рекурсию.
    """
    # Создаём словарь для O(1) доступа: category_id -> parent_id
    # Это быстрее, чем искать в DataFrame на каждой итерации
    cat_to_parent = dict(
        zip(
            category_tree_df.select("category_id").to_series().to_list(),
            category_tree_df.select("parent_id").to_series().to_list()
        )
    )
    
    # Кеш для уже вычисленных глубин (мемоизация)
    # Позволяет избежать повторных вычислений и обработать циклы
    depth_cache = {}
    
    def get_depth(cat_id):
        """Рекурсивная функция для вычисления глубины категории.
        
        Алгоритм:
        1. Если категория в кеше - возвращаем сразу
        2. Если parentid is None - это корень (уровень 1)
        3. Иначе рекурсивно вычисляем глубину родителя + 1
        """
        if cat_id in depth_cache:
            return depth_cache[cat_id]
        
        parent_id = cat_to_parent.get(cat_id)
        
        if parent_id is None:
            # Корневая категория (уровень 1)
            depth = 1
        else:
            # Рекурсивно вычисляем глубину родителя и добавляем 1
            depth = get_depth(parent_id) + 1
        
        depth_cache[cat_id] = depth
        return depth
    
    # Вычисляем глубину для всех категорий
    depths = []
    for cat_id in category_tree_df.select("category_id").to_series().to_list():
        depths.append({"category_id": cat_id, "level": get_depth(cat_id)})
    
    return pl.DataFrame(depths)

# Применяем рекурсивный анализ ко всему дереву категорий
category_levels = category_tree.join(
    calculate_depth(category_tree),
    on="category_id",
    how="left",
    coalesce=True
)

logger.info("СТАТИСТИКА ПО УРОВНЯМ (РЕКУРСИВНЫЙ АНАЛИЗ):")
level_stats = category_levels.group_by("level").agg(pl.len().alias("count")).sort("level")
logger.info(f"{level_stats}")

max_depth = category_levels.select(pl.col("level").max()).item()
avg_depth = category_levels.select(pl.col("level").mean()).item()
logger.info("Глубина дерева:")
logger.info(f"   Максимальная: {max_depth} уровней")
logger.info(f"   Средняя: {avg_depth:.2f} уровня")

# Примеры категорий с их уровнями
logger.info("ПРИМЕРЫ КАТЕГОРИЙ:")
logger.info(f"{category_tree.join(category_levels.select(['category_id', 'level']), on='category_id', how='left', coalesce=True).head(10)}")

2025-12-21 17:26:47 - INFO - Всего категорий в дереве: 1668
2025-12-21 17:26:47 - INFO - Корневых категорий (без родителя): 25
2025-12-21 17:26:47 - INFO - СТАТИСТИКА ПО УРОВНЯМ (РЕКУРСИВНЫЙ АНАЛИЗ):
2025-12-21 17:26:47 - INFO - shape: (6, 2)
┌───────┬───────┐
│ level ┆ count │
│ ---   ┆ ---   │
│ i64   ┆ u32   │
╞═══════╪═══════╡
│ 1     ┆ 25    │
│ 2     ┆ 174   │
│ 3     ┆ 701   │
│ 4     ┆ 665   │
│ 5     ┆ 90    │
│ 6     ┆ 13    │
└───────┴───────┘
2025-12-21 17:26:47 - INFO - Глубина дерева:
2025-12-21 17:26:47 - INFO -    Максимальная: 6 уровней
2025-12-21 17:26:47 - INFO -    Средняя: 3.40 уровня
2025-12-21 17:26:47 - INFO - ПРИМЕРЫ КАТЕГОРИЙ:
2025-12-21 17:26:47 - INFO - shape: (10, 3)
┌─────────────┬───────────┬───────┐
│ category_id ┆ parent_id ┆ level │
│ ---         ┆ ---       ┆ ---   │
│ i64         ┆ i64       ┆ i64   │
╞═════════════╪═══════════╪═══════╡
│ 176         ┆ 713       ┆ 4     │
│ 328         ┆ 54        ┆ 4     │
│ 671         ┆ 1426      ┆ 3     │
│ 1458 

### Анализ глубины дерева категорий

**Структура дерева:**
- **1,668 категорий** в дереве
- **25 корневых категорий** (без родителя)
- **Максимальная глубина: 6 уровней** - дерево достаточно глубокое
- **Средняя глубина: 3.40 уровня** - большинство категорий на уровнях 3-4

**Распределение по уровням:**
- Уровень 1 (корневые): 25 категорий
- Уровень 2: 174 категории
- Уровень 3: 701 категория (самый большой уровень)
- Уровень 4: 665 категорий
- Уровень 5: 90 категорий
- Уровень 6: 13 категорий (листья)

**Критический вывод:**
- **Рекурсия НУЖНА!** Упрощённый анализ (только 2 уровня) неверен
- Дерево имеет до 6 уровней вложенности, а не 2
- Большинство категорий находятся на уровнях 3-4

**Применение для модели:**
1. **Использовать иерархию для content-based рекомендаций:**
   - Рекомендовать товары из той же подкатегории
   - Использовать родительские категории для обобщения
   - Учитывать близость в дереве (общие предки)
2. **Re-ranking:** предпочитать товары из той же ветки категорий
3. **Cold-start:** использовать корневые категории для начальных рекомендаций



## 5. Интеграция данных: события + свойства товаров + категории

### 5.1. Покрытие товаров свойствами и категориями


In [8]:
# ИНТЕГРАЦИЯ ДАННЫХ: АНАЛИЗ ПОКРЫТИЯ МЕТАДАННЫМИ
# Критически важно понять, какая доля товаров из событий имеет метаданные
# Это определяет, насколько эффективны будут content-based подходы

# ТОВАРЫ ИЗ СОБЫТИЙ (те, с которыми взаимодействовали пользователи)
items_in_events = events.select("item_id").unique()
n_items_in_events = items_in_events.n_unique()

# ТОВАРЫ ИЗ СВОЙСТВ (те, у которых есть метаданные)
items_in_props = item_properties.select("item_id").unique()
n_items_in_props = items_in_props.n_unique()

# ПЕРЕСЕЧЕНИЕ: товары, которые есть И в событиях, И в свойствах
# Используем join вместо is_in для совместимости с Polars 1.36+ (быстрее и без warnings)
items_in_both = items_in_events.join(
    items_in_props,
    on="item_id",
    how="inner"  # Только общие товары
).n_unique()

logger.info("ПОКРЫТИЕ ТОВАРОВ:")
logger.info(f"   Товаров в событиях: {n_items_in_events:,}")
logger.info(f"   Товаров со свойствами: {n_items_in_props:,}")
logger.info(f"   Товаров и в событиях, и со свойствами: {items_in_both:,}")
logger.info(f"   Покрытие: {items_in_both / n_items_in_events * 100:.2f}%")

# ТОВАРЫ С КАТЕГОРИЯМИ (подмножество товаров со свойствами)
if len(categories_from_props) > 0:
    items_with_cats = categories_from_props.select("item_id").unique()
    # Пересечение: товары из событий, у которых есть категории
    items_in_events_with_cats = items_in_events.join(
        items_with_cats,
        on="item_id",
        how="inner"
    ).n_unique()
    
    logger.info("ПОКРЫТИЕ КАТЕГОРИЯМИ:")
    logger.info(f"   Товаров с категориями: {items_with_cats.n_unique():,}")
    logger.info(f"   Товаров в событиях с категориями: {items_in_events_with_cats:,}")
    logger.info(f"   Покрытие: {items_in_events_with_cats / n_items_in_events * 100:.2f}%")

2025-12-21 17:26:47 - INFO - ПОКРЫТИЕ ТОВАРОВ:
2025-12-21 17:26:47 - INFO -    Товаров в событиях: 235,061
2025-12-21 17:26:47 - INFO -    Товаров со свойствами: 417,052
2025-12-21 17:26:47 - INFO -    Товаров и в событиях, и со свойствами: 185,246
2025-12-21 17:26:47 - INFO -    Покрытие: 78.81%
2025-12-21 17:26:47 - INFO - ПОКРЫТИЕ КАТЕГОРИЯМИ:
2025-12-21 17:26:47 - INFO -    Товаров с категориями: 417,052
2025-12-21 17:26:47 - INFO -    Товаров в событиях с категориями: 185,246
2025-12-21 17:26:47 - INFO -    Покрытие: 78.81%


### Анализ покрытия метаданными

**Покрытие свойствами:**
- **78.81% товаров** из событий имеют метаданные - **ХОРОШО**, но не идеально
- **21.19% товаров в событиях БЕЗ метаданных** - нужен fallback для них

**Покрытие категориями:**
- **78.81% товаров** из событий имеют категории
- Покрытие категориями **равно покрытию свойствами** - все товары со свойствами имеют категории!

**Выводы:**
1. **Content-based подходы можно применять к 78.81% товаров** из событий
2. Для остальных **21.19% товаров** - только collaborative filtering или popular fallback
3. **Категории доступны для всех товаров с метаданными** - можно использовать для content-based рекомендаций

**Применение для модели:**
- Content-based рекомендации по категориям возможны для **78.81% товаров**
- Для товаров без метаданных использовать ALS или popular fallback
- Re-ranking с учётом категорий для улучшения качества рекомендаций



### 5.2. Статистика по категориям в событиях


In [9]:
# Статистика по категориям в событиях
# ВАЖНО: item_properties содержит историю изменений (товар мог иметь несколько категорий)
# Берём последнюю категорию каждого товара (max timestamp) для корректного join
if len(categories_from_props) > 0:
    # Получаем последнюю категорию каждого товара (актуальную на момент последнего изменения)
    # Это необходимо, так как у 23,352 товаров есть несколько категорий в истории
    item_properties_with_ts = item_properties.filter(
        pl.col("property") == "categoryid"
    ).select([
        pl.col("item_id"),
        pl.col("value").cast(pl.Int64).alias("categoryid"),
        pl.col("timestamp"),
    ])
    
    # Берём последнюю категорию для каждого товара
    latest_categories = item_properties_with_ts.sort(["item_id", "timestamp"], descending=[False, True]).group_by("item_id").first().select(["item_id", "categoryid"])
    
    # Join с событиями (теперь каждый товар имеет только одну категорию)
    events_with_cats = events.join(
        latest_categories,
        on="item_id",
        how="left",
        coalesce=True
    )
    
    logger.info("СОБЫТИЯ ПО КАТЕГОРИЯМ:")
    n_events_with_cats_count = events_with_cats.filter(pl.col('categoryid').is_not_null()).height
    logger.info(f"   Событий с категориями: {n_events_with_cats_count:,}")
    logger.info(f"   (Покрытие: {n_events_with_cats_count / len(events) * 100:.2f}%)")
    
    # Топ категорий по событиям
    top_cat_events = events_with_cats.filter(
        pl.col("categoryid").is_not_null()
    ).group_by("categoryid").agg([
        pl.len().alias("n_events"),
        pl.n_unique("user_id").alias("n_users"),
        pl.n_unique("item_id").alias("n_items"),
    ]).sort("n_events", descending=True)
    
    logger.info("ТОП-10 КАТЕГОРИЙ ПО КОЛИЧЕСТВУ СОБЫТИЙ:")
    logger.info(f"{top_cat_events.head(10)}")

2025-12-21 17:26:48 - INFO - СОБЫТИЯ ПО КАТЕГОРИЯМ:
2025-12-21 17:26:48 - INFO -    Событий с категориями: 2,500,062
2025-12-21 17:26:48 - INFO -    (Покрытие: 90.73%)
2025-12-21 17:26:48 - INFO - ТОП-10 КАТЕГОРИЙ ПО КОЛИЧЕСТВУ СОБЫТИЙ:
2025-12-21 17:26:48 - INFO - shape: (10, 4)
┌────────────┬──────────┬─────────┬─────────┐
│ categoryid ┆ n_events ┆ n_users ┆ n_items │
│ ---        ┆ ---      ┆ ---     ┆ ---     │
│ i64        ┆ u32      ┆ u32     ┆ u32     │
╞════════════╪══════════╪═════════╪═════════╡
│ 1051       ┆ 75208    ┆ 36191   ┆ 2557    │
│ 1483       ┆ 64774    ┆ 40778   ┆ 2945    │
│ 491        ┆ 61109    ┆ 38029   ┆ 2417    │
│ 959        ┆ 52277    ┆ 27166   ┆ 1890    │
│ 342        ┆ 46847    ┆ 26264   ┆ 4697    │
│ 683        ┆ 38883    ┆ 15962   ┆ 939     │
│ 1279       ┆ 34374    ┆ 15219   ┆ 1023    │
│ 5          ┆ 29714    ┆ 15103   ┆ 288     │
│ 646        ┆ 28240    ┆ 13537   ┆ 1381    │
│ 196        ┆ 27683    ┆ 16598   ┆ 1659    │
└────────────┴──────────┴────

### Анализ популярности категорий

**События с категориями:**
- **2,500,062 уникальных событий** имеют категории (90.73% покрытие)
- **Примечание:** В `item_properties` содержится история изменений категорий - у некоторых товаров было несколько категорий во времени. Для корректного подсчёта используется последняя категория каждого товара (max timestamp).
- Это позволяет анализировать, какие категории наиболее интересны пользователям

**Топ категории:**
- Категория **1051**: 75,208 событий, 36,191 пользователей (лидер по событиям)
- Категория **1483**: 64,774 событий, 40,778 пользователей (лидер по пользователям)
- Категория **491**: 61,109 событий, 38,029 пользователей
- Значения топ категорий корректны, так как считаются после join с последней категорией товара

**Выводы:**
1. **Неравномерное распределение** - есть мега-популярные категории (1051: 75K событий!)
2. Топ-10 категорий сильно выделяются по количеству событий
3. Некоторые категории привлекают много пользователей (1483: 40,778 пользователей, 491: 38,029 пользователей)

**Применение для модели:**
1. **Cold-start fallback:** использовать популярные категории (топ-10) для новых пользователей
2. **Re-ranking:** учитывать популярность категории пользователя при ранжировании
3. **Diversity constraint:** ограничивать количество товаров из одной категории в рекомендациях
4. **Категорийный fallback:** для пользователей с известной категорией → рекомендовать популярные товары из этой категории



## 6. Ключевые выводы и рекомендации

### Основные находки:
1. **Разреженность данных**: Очень высокая - median 1 событие на пользователя
2. **Покрытие метаданными**: Необходимо проверить покрытие товаров свойствами и категориями
3. **Категории**: Доступны и могут использоваться для content-based подходов
4. **Свойства товаров**: Множество различных свойств, требуется feature engineering

### Рекомендации для модели:
1. **Использовать категории** для улучшения рекомендаций:
   - Content-based фильтрация по категориям
   - Re-ranking с учётом категорий
   - Cold-start стратегия через популярные категории

2. **Feature engineering из свойств**:
   - Извлечь основные свойства (бренд, цена, доступность)
   - Создать embeddings категорий
   - Использовать для hybrid рекомендаций

3. **Hybrid подход**:
   - ALS для warm users (collaborative filtering)
   - Content-based для cold items/ cold users
   - Popular items fallback
